In [ ]:
# -*- coding: utf-8 -*-
"""Untitled8.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1InNNgGKMDBwqxiElvwp00cYiaOOJLBLi
"""



"""# Task
Create a Python application that simulates an algo trading platform similar to Groww, using a provided dataset for backtesting trading strategies.

## Load and preprocess data

### Subtask:
Load the provided dataset and prepare it for analysis. This may involve handling missing values, formatting dates, and selecting relevant columns.

**Reasoning**:
Load the Excel file into a pandas DataFrame, display the first 5 rows, and show the DataFrame's information to understand its structure and data types.
"""

import pandas as pd

# Load the Excel file
df = pd.read_excel('/content/Backtesting Report.xlsx')

# Display the first 5 rows
display(df.head())

# Display DataFrame information
display(df.info())

"""**Reasoning**:
Based on the `df.info()` output, there are no missing values. The 'Date/Time' column is already in datetime format. Now select the relevant columns for backtesting. The relevant columns appear to be 'Date/Time' and 'Price INR'. The other columns represent results of a backtest, not input data.


"""

# Select relevant columns
df_selected = df[['Date/Time', 'Price INR']].copy()

# Display the first 5 rows of the selected columns
display(df_selected.head())

"""## Develop trading strategy

### Subtask:
Define the trading algorithm or strategy that the app will use. This could be a simple rule-based strategy or a more complex machine learning model.

**Reasoning**:
Define the moving average crossover strategy by calculating the short-term and long-term moving averages and generating trading signals based on their crossover points.
"""

# Define short and long window lengths
short_window = 40
long_window = 100

# Calculate moving averages
df_selected['Short_MA'] = df_selected['Price INR'].rolling(window=short_window).mean()
df_selected['Long_MA'] = df_selected['Price INR'].rolling(window=long_window).mean()

# Generate signals
df_selected['Signal'] = 0.0
# Use .loc to avoid SettingWithCopyWarning
df_selected.loc[short_window:, 'Signal'] = (df_selected['Short_MA'][short_window:] > df_selected['Long_MA'][short_window:]).astype(float)

# Generate positions
df_selected['Position'] = df_selected['Signal'].diff()

# Display the first few rows with the new columns
display(df_selected.head())

"""## Implement backtesting engine

### Subtask:
Create a system to test the trading strategy on historical data. This will involve simulating trades and calculating performance metrics.

**Reasoning**:
Implement the trading simulation logic to track capital, shares, and calculate the final profit/loss based on the generated trading signals in `df_selected`.
"""

initial_capital = 100000  # Initial capital in INR
capital = initial_capital
shares_held = 0

# Find the first row where 'Position' is not NaN to start the simulation
start_index = df_selected['Position'].first_valid_index()

if start_index is not None:
    for index, row in df_selected.loc[start_index:].iterrows():
        if row['Position'] == 1.0:  # Buy signal
            # Calculate the number of shares to buy
            # Buy as many shares as possible without exceeding capital
            shares_to_buy = int(capital / row['Price INR'])
            buy_cost = shares_to_buy * row['Price INR']

            # Ensure we don't exceed available capital
            if buy_cost <= capital:
                capital -= buy_cost
                shares_held += shares_to_buy

        elif row['Position'] == -1.0:  # Sell signal
            # Sell all held shares
            sell_revenue = shares_held * row['Price INR']
            capital += sell_revenue
            shares_held = 0

    # Calculate the final portfolio value
    final_portfolio_value = capital + (shares_held * df_selected.iloc[-1]['Price INR'])

    # Calculate total profit or loss
    total_profit_loss = final_portfolio_value - initial_capital

    # Print the results
    print(f"Initial Capital: {initial_capital:.2f} INR")
    print(f"Final Portfolio Value: {final_portfolio_value:.2f} INR")
    print(f"Total Profit/Loss: {total_profit_loss:.2f} INR")
else:
    print("No valid trading positions found in the DataFrame.")

"""## Backtest the strategy

### Subtask:
Run the backtesting engine with the developed strategy and the loaded data to evaluate its performance.

## Analyze backtesting results

### Subtask:
Analyze the results of the backtest to understand the strategy's profitability, risk, and other key metrics.

**Reasoning**:
Calculate the percentage return and assess the profitability based on the backtesting results.
"""

# Calculate percentage return
percentage_return = (total_profit_loss / initial_capital) * 100

print(f"Percentage Return: {percentage_return:.2f}%")

# Assess profitability
if total_profit_loss > 0:
    profitability_assessment = "The strategy appears to be profitable."
elif total_profit_loss < 0:
    profitability_assessment = "The strategy appears to be unprofitable."
else:
    profitability_assessment = "The strategy broke even."

print(f"Profitability Assessment: {profitability_assessment}")

# Document initial observations
print("\nInitial Observations:")
print(f"- The trading strategy resulted in a total profit/loss of {total_profit_loss:.2f} INR over the backtesting period.")
print(f"- This corresponds to a percentage return of {percentage_return:.2f}%.")
print(f"- Based on these high-level metrics, the strategy is currently assessed as {profitability_assessment.lower().replace('the strategy appears to be ', '')}.")
print("- Further analysis of risk metrics and trade details is needed for a comprehensive evaluation.")

"""## Visualize results

### Subtask:
Create visualizations to represent the backtesting results, such as performance charts, drawdowns, and trade logs.

**Reasoning**:
The next steps involve plotting the portfolio value and drawdown over time, which requires importing matplotlib.pyplot and numpy, calculating the portfolio value and drawdown, and then generating the plots.
"""

import matplotlib.pyplot as plt
import numpy as np

# Calculate Portfolio Value
# Use the refined strategy results for plotting
df_selected['Portfolio_Value_refined'] = 0.0
# Find the first valid index for the refined strategy's positions
start_index_refined = df_selected['Position_refined'].first_valid_index()


if start_index_refined is not None:
    df_selected.loc[start_index_refined, 'Portfolio_Value_refined'] = initial_capital

    capital_refined_plot = initial_capital # Use a separate variable for plotting
    shares_held_refined_plot = 0 # Use a separate variable for plotting

    for index in range(start_index_refined + 1, len(df_selected)):
        row = df_selected.iloc[index]
        prev_row = df_selected.iloc[index - 1]

        # Re-calculate capital and shares held based on the previous row's state and current row's signal for the refined strategy
        if prev_row['Position_refined'] == 1.0:  # If there was a buy signal in the previous row for refined strategy
            shares_to_buy_refined_plot = int(capital_refined_plot / prev_row['Price INR'])
            buy_cost_refined_plot = shares_to_buy_refined_plot * prev_row['Price INR']
            if buy_cost_refined_plot <= capital_refined_plot:
                capital_refined_plot -= buy_cost_refined_plot
                shares_held_refined_plot += shares_to_buy_refined_plot

        elif prev_row['Position_refined'] == -1.0:  # If there was a sell signal in the previous row for refined strategy
            sell_revenue_refined_plot = shares_held_refined_plot * prev_row['Price INR']
            capital_refined_plot += sell_revenue_refined_plot
            shares_held_refined_plot = 0


        # Calculate portfolio value for the current row using refined strategy
        if shares_held_refined_plot > 0:
            df_selected.loc[row.name, 'Portfolio_Value_refined'] = capital_refined_plot + (shares_held_refined_plot * row['Price INR'])
        else:
            df_selected.loc[row.name, 'Portfolio_Value_refined'] = capital_refined_plot

    # Plot Portfolio Value for refined strategy
    fig1, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(df_selected['Date/Time'], df_selected['Portfolio_Value_refined'], label='Refined Portfolio Value')
    ax1.set_title('Refined Portfolio Value Over Time')
    ax1.set_xlabel('Date/Time')
    ax1.set_ylabel('Portfolio Value (INR)')
    ax1.legend()
    ax1.grid(True)

    # Add cursor functionality to Portfolio Value plot
    def motion_portfolio(event):
        if event.inaxes == ax1:
            x, y = event.xdata, event.ydata
            ax1.set_title(f'Refined Portfolio Value Over Time\nDate/Time: {plt.matplotlib.dates.num2date(x).strftime("%Y-%m-%d %H:%M:%S")}, Value: {y:.2f} INR')
        else:
            ax1.set_title('Refined Portfolio Value Over Time')
    fig1.canvas.mpl_connect('motion_notify_event', motion_portfolio)


    # Calculate Drawdown for refined strategy
    df_selected['Peak_refined'] = df_selected['Portfolio_Value_refined'].cummax()
    df_selected['Drawdown_refined'] = (df_selected['Portfolio_Value_refined'] - df_selected['Peak_refined']) / df_selected['Peak_refined'] * 100

    # Plot Drawdown for refined strategy
    fig2, ax2 = plt.subplots(figsize=(12, 6))
    ax2.plot(df_selected['Date/Time'], df_selected['Drawdown_refined'], label='Refined Drawdown', color='red')
    ax2.set_title('Refined Portfolio Drawdown Over Time')
    ax2.set_xlabel('Date/Time')
    ax2.set_ylabel('Drawdown (%)')
    ax2.legend()
    ax2.grid(True)

    # Add cursor functionality to Drawdown plot
    def motion_drawdown(event):
        if event.inaxes == ax2:
            x, y = event.xdata, event.ydata
            ax2.set_title(f'Refined Portfolio Drawdown Over Time\nDate/Time: {plt.matplotlib.dates.num2date(x).strftime("%Y-%m-%d %H:%M:%S")}, Drawdown: {y:.2f}%')
        else:
            ax2.set_title('Refined Portfolio Drawdown Over Time')
    fig2.canvas.mpl_connect('motion_notify_event', motion_drawdown)

    plt.show()
else:
    print("No valid trading positions found for the refined strategy to plot.")

"""## Summary:

### Data Analysis Key Findings

* The dataset was loaded successfully with no missing values and the 'Date/Time' column already in the correct datetime format.
* A simple moving average crossover strategy (40-day short window, 100-day long window) was initially implemented to generate trading signals.
* Backtesting with an initial capital of 100,000 INR resulted in a final portfolio value of 61,006.00 INR.
* The total profit/loss from the initial backtest was -38,994.00 INR, representing a percentage return of -38.99%.
* The initial strategy was assessed as unprofitable based on the backtesting results.
* Visualizations were generated showing the portfolio value over time and the portfolio drawdown over time, with cursor functionality added for detailed analysis.
* A visualization showing the price with buy and sell signals was also generated to provide a live tracking view of the trading performance.

### Refined Strategy and Results

* The strategy was refined by changing the moving average window lengths (20-day short window, 50-day long window).
* Rerunning the backtest with the refined strategy resulted in a final portfolio value of 132,742.50 INR.
* The total profit/loss from the refined strategy's backtest was 32,742.50 INR, representing a percentage return of 32.74%.
* The refined strategy appears to be profitable based on these backtesting results.
* Visualizations of the refined strategy's portfolio value and drawdown were also generated with cursor functionality.

### Insights or Next Steps

* The refinement of the moving average window lengths significantly improved the strategy's performance, turning an unprofitable strategy into a profitable one for the backtesting period.
* This highlights the importance of optimizing strategy parameters.
* Implementing more sophisticated risk management techniques (e.g., stop-loss orders, take-profit orders) and calculating additional performance metrics (e.g., Sharpe Ratio, Sortino Ratio, maximum drawdown duration, Calmar Ratio) would provide a more comprehensive evaluation of the refined strategy.
* Exploring other trading strategies or incorporating machine learning models could potentially lead to even better performance.
* For a more complete trading application, building a user interface to interact with the strategy and visualize results in real-time or on historical data would be beneficial.

## Summary:

### Data Analysis Key Findings

* The dataset was loaded successfully with no missing values and the 'Date/Time' column already in the correct datetime format.
* A simple moving average crossover strategy (40-day short window, 100-day long window) was initially implemented to generate trading signals.
* Backtesting with an initial capital of 100,000 INR resulted in a final portfolio value of 61,006.00 INR.
* The total profit/loss from the initial backtest was -38,994.00 INR, representing a percentage return of -38.99%.
* The initial strategy was assessed as unprofitable based on the backtesting results.


### Refined Strategy and Results

* The strategy was refined by changing the moving average window lengths (20-day short window, 50-day long window).
* Rerunning the backtest with the refined strategy resulted in a final portfolio value of 132,742.50 INR.
* The total profit/loss from the refined strategy's backtest was 32,742.50 INR, representing a percentage return of 32.74%.
* The refined strategy appears to be profitable based on these backtesting results.

### Detailed Refined Strategy Performance Metrics

* **Sharpe Ratio (Refined Strategy):** (Value calculated in the cell below)
* **Maximum Drawdown Duration (Refined Strategy):** (Value calculated in the cell below)
* **Win Rate (Refined Strategy):** (Value calculated in the cell below)
* **Loss Rate (Refined Strategy):** (Value calculated in the cell below)
* **Average Win (Refined Strategy):** (Value calculated in the cell below)
* **Average Loss (Refined Strategy):** (Value calculated in the cell below)

### Visualizations

* Visualizations were generated showing the initial strategy's portfolio value over time and the portfolio drawdown over time, with cursor functionality added for detailed analysis.
* Visualizations of the refined strategy's portfolio value (Equity Curve) and drawdown were also generated with cursor functionality.
* A visualization showing the price with buy and sell signals for the refined strategy was generated to provide a view of the trading performance relative to price movements, also with cursor functionality.


### Insights or Next Steps

* The refinement of the moving average window lengths significantly improved the strategy's performance, turning an unprofitable strategy into a profitable one for the backtesting period.
* This highlights the importance of optimizing strategy parameters.
* Implementing more sophisticated risk management techniques (e.g., stop-loss orders, take-profit orders) and calculating additional performance metrics could further enhance the strategy and its evaluation.
* Exploring other trading strategies or incorporating machine learning models could potentially lead to even better performance.
* For a more complete trading application experience with a dynamic dashboard, building a user interface to interact with the strategy and visualize results in real-time or on historical data would be beneficial, but this is beyond the scope of this notebook environment.

## Analyze Refined Strategy Results (Detailed Metrics)

### Subtask:
Calculate and display additional detailed performance and risk metrics for the refined strategy.

**Reasoning**:
Calculate Sharpe Ratio, Maximum Drawdown Duration, Win Rate, Loss Rate, and Average Win/Loss for the refined strategy to provide a more comprehensive analysis of its performance.
"""

# Calculate Sharpe Ratio
# Requires daily returns and risk-free rate (assuming risk-free rate is 0 for simplicity)
# Use the existing 'Portfolio_Value_refined' column calculated in the previous plotting step.

# Ensure 'Portfolio_Value_refined' column exists
if 'Portfolio_Value_refined' in df_selected.columns:
    df_selected['Daily_Return_refined'] = df_selected['Portfolio_Value_refined'].pct_change()

    # Replace infinite values with NaN and drop NaN values from daily returns before calculating std
    daily_returns_cleaned = df_selected['Daily_Return_refined'].replace([np.inf, -np.inf], np.nan).dropna()

    # Assuming 252 trading days in a year
    # Handle cases where std is 0 or NaN to avoid division by zero or NaN
    if daily_returns_cleaned.std() is not None and not np.isnan(daily_returns_cleaned.std()) and daily_returns_cleaned.std() != 0:
        sharpe_ratio_refined = (daily_returns_cleaned.mean() / daily_returns_cleaned.std()) * np.sqrt(252)
        print(f"Sharpe Ratio (Refined Strategy): {sharpe_ratio_refined:.2f}")
    else:
        print("Sharpe Ratio (Refined Strategy): Cannot be calculated (standard deviation is zero or NaN)")
else:
    print("Sharpe Ratio (Refined Strategy): Cannot be calculated. 'Portfolio_Value_refined' column not found.")


# Calculate Maximum Drawdown Duration
# This requires finding the longest period between a peak and a new peak
# Use the existing 'Portfolio_Value_refined' column
if 'Portfolio_Value_refined' in df_selected.columns:
    df_selected['Peak_refined'] = df_selected['Portfolio_Value_refined'].cummax()
    df_selected['Drawdown_refined_value'] = df_selected['Portfolio_Value_refined'] - df_selected['Peak_refined']

    # Find periods of drawdown
    in_drawdown = df_selected['Drawdown_refined_value'] < 0
    # Find the start of drawdown periods
    drawdown_starts = df_selected.index[in_drawdown & (~in_drawdown.shift(1).fillna(False))]
    # Find the end of drawdown periods
    drawdown_ends = df_selected.index[~in_drawdown & (in_drawdown.shift(1).fillna(False))]

    # Handle case where last period is a drawdown
    if in_drawdown.iloc[-1]:
        drawdown_ends = drawdown_ends.append(pd.Index([df_selected.index[-1]]))

    drawdown_durations = [df_selected['Date/Time'][end] - df_selected['Date/Time'][start] for start, end in zip(drawdown_starts, drawdown_ends)]

    max_drawdown_duration_refined = max(drawdown_durations) if drawdown_durations else pd.Timedelta(seconds=0)


    print(f"Maximum Drawdown Duration (Refined Strategy): {max_drawdown_duration_refined}")
else:
     print("Maximum Drawdown Duration (Refined Strategy): Cannot be calculated. 'Portfolio_Value_refined' column not found.")


# Calculate Win Rate and Loss Rate, and Average Win/Loss
# Need to identify individual trades from the 'Position_refined' column
if 'Position_refined' in df_selected.columns:
    trades = []
    entry_price = None
    entry_date = None

    # Iterate through the DataFrame to identify trades based on Position_refined
    for index, row in df_selected.loc[df_selected['Position_refined'] != 0].iterrows():
        if row['Position_refined'] == 1.0:  # Buy signal
            # If we are not already in a position (entry_price is None)
            if entry_price is None:
                entry_price = row['Price INR']
                entry_date = row['Date/Time']
        elif row['Position_refined'] == -1.0: # Sell signal
            # If we are in a position (entry_price is not None)
            if entry_price is not None:
                exit_price = row['Price INR']
                exit_date = row['Date/Time']
                # Assuming a single unit trade for PnL calculation
                pnl = (exit_price - entry_price)
                trades.append({'Entry_Date': entry_date, 'Exit_Date': exit_date, 'Entry_Price': entry_price, 'Exit_Price': exit_price, 'PnL': pnl})
                # Reset for the next trade
                entry_price = None

    # Handle an open position at the end of the data
    if entry_price is not None:
         exit_price = df_selected.iloc[-1]['Price INR']
         exit_date = df_selected.iloc[-1]['Date/Time']
         pnl = (exit_price - entry_price)
         trades.append({'Entry_Date': entry_date, 'Exit_Date': exit_date, 'Entry_Price': entry_price, 'Exit_Price': exit_price, 'PnL': pnl, 'Status': 'Open'})


    trades_df = pd.DataFrame(trades)

    if not trades_df.empty:
        winning_trades = trades_df[trades_df['PnL'] > 0]
        losing_trades = trades_df[trades_df['PnL'] < 0]

        win_rate_refined = (len(winning_trades) / len(trades_df)) * 100
        loss_rate_refined = (len(losing_trades) / len(trades_df)) * 100

        average_win_refined = winning_trades['PnL'].mean() if not winning_trades.empty else 0
        average_loss_refined = losing_trades['PnL'].mean() if not losing_trades.empty else 0

        print(f"Win Rate (Refined Strategy): {win_rate_refined:.2f}%")
        print(f"Loss Rate (Refined Strategy): {loss_rate_refined:.2f}%")
        print(f"Average Win (Refined Strategy): {average_win_refined:.2f} INR")
        print(f"Average Loss (Refined Strategy): {average_loss_refined:.2f} INR")
    else:
        print("No completed trades found for calculating win/loss metrics.")
else:
    print("Win Rate, Loss Rate, and Average Win/Loss: Cannot be calculated. 'Position_refined' column not found.")

"""**Reasoning**:
Add an equity curve plot to visualize the growth of the portfolio value over time, which is a key metric in evaluating trading strategy performance.
"""

# Plot Equity Curve (which is essentially the Portfolio Value plot)
# We already have the Portfolio Value calculated, so we can just plot it again with a different title if desired.

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df_selected['Date/Time'], df_selected['Portfolio_Value_refined'], label='Equity Curve (Refined Strategy)') # Using refined strategy results
ax.set_title('Equity Curve Over Time (Refined Strategy)')
ax.set_xlabel('Date/Time')
ax.set_ylabel('Portfolio Value (INR)')
ax.legend()
ax.grid(True)

# Add cursor functionality to Equity Curve plot
def motion_equity(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        ax.set_title(f'Equity Curve Over Time (Refined Strategy)\nDate/Time: {plt.matplotlib.dates.num2date(x).strftime("%Y-%m-%d %H:%M:%S")}, Value: {y:.2f} INR')
    else:
        ax.set_title('Equity Curve Over Time (Refined Strategy)')
fig.canvas.mpl_connect('motion_notify_event', motion_equity)

plt.show()

"""**Reasoning**:
Visualize the price data along with the buy and sell signals generated by the refined strategy to get a live tracking view of the algo trading performance.
"""

# Plotting price with buy/sell signals for refined strategy
fig, ax = plt.subplots(figsize=(14, 7))

# Plot the price data
ax.plot(df_selected['Date/Time'], df_selected['Price INR'], label='Price INR', alpha=0.5)

# Plot buy signals
ax.plot(df_selected['Date/Time'][df_selected['Position_refined'] == 1.0],
        df_selected['Price INR'][df_selected['Position_refined'] == 1.0],
        '^', markersize=10, color='g', lw=0, label='Buy Signal')

# Plot sell signals
ax.plot(df_selected['Date/Time'][df_selected['Position_refined'] == -1.0],
        df_selected['Price INR'][df_selected['Position_refined'] == -1.0],
        'v', markersize=10, color='r', lw=0, label='Sell Signal')

# Add titles and labels
ax.set_title('Price and Trading Signals (Refined Strategy)')
ax.set_xlabel('Date/Time')
ax.set_ylabel('Price INR')
ax.legend()
ax.grid(True)

# Add cursor functionality
def motion_signals(event):
    if event.inaxes == ax:
        x, y = event.xdata, event.ydata
        ax.set_title(f'Price and Trading Signals (Refined Strategy)\nDate/Time: {plt.matplotlib.dates.num2date(x).strftime("%Y-%m-%d %H:%M:%S")}, Price: {y:.2f} INR')
    else:
        ax.set_title('Price and Trading Signals (Refined Strategy)')

fig.canvas.mpl_connect('motion_notify_event', motion_signals)

plt.show()

"""## Refine strategy

### Subtask:
Refine or modify the trading strategy to improve its performance based on the backtesting results.

**Reasoning**:
Based on the initial unprofitable results, let's try refining the strategy by allowing the user to adjust the short and long moving average window lengths and re-running the backtest.
"""

# Refine strategy by changing window lengths
short_window_refined = 20  # Example: change short window
long_window_refined = 50   # Example: change long window

# Calculate new moving averages
df_selected['Short_MA_refined'] = df_selected['Price INR'].rolling(window=short_window_refined).mean()
df_selected['Long_MA_refined'] = df_selected['Price INR'].rolling(window=long_window_refined).mean()

# Generate new signals
df_selected['Signal_refined'] = 0.0
df_selected.loc[long_window_refined:, 'Signal_refined'] = (df_selected['Short_MA_refined'][long_window_refined:] > df_selected['Long_MA_refined'][long_window_refined:]).astype(float)

# Generate new positions
df_selected['Position_refined'] = df_selected['Signal_refined'].diff()

# Rerun backtesting engine with refined strategy
capital_refined = initial_capital
shares_held_refined = 0

start_index_refined = df_selected['Position_refined'].first_valid_index()

if start_index_refined is not None:
    for index, row in df_selected.loc[start_index_refined:].iterrows():
        if row['Position_refined'] == 1.0:  # Buy signal
            shares_to_buy_refined = int(capital_refined / row['Price INR'])
            buy_cost_refined = shares_to_buy_refined * row['Price INR']
            if buy_cost_refined <= capital_refined:
                capital_refined -= buy_cost_refined
                shares_held_refined += shares_to_buy_refined

        elif row['Position_refined'] == -1.0:  # Sell signal
            sell_revenue_refined = shares_held_refined * row['Price INR']
            capital_refined += sell_revenue_refined
            shares_held_refined = 0

    final_portfolio_value_refined = capital_refined + (shares_held_refined * df_selected.iloc[-1]['Price INR'])
    total_profit_loss_refined = final_portfolio_value_refined - initial_capital
    percentage_return_refined = (total_profit_loss_refined / initial_capital) * 100

    print(f"Refined Strategy Results (Short Window: {short_window_refined}, Long Window: {long_window_refined}):")
    print(f"Initial Capital: {initial_capital:.2f} INR")
    print(f"Final Portfolio Value: {final_portfolio_value_refined:.2f} INR")
    print(f"Total Profit/Loss: {total_profit_loss_refined:.2f} INR")
    print(f"Percentage Return: {percentage_return_refined:.2f}%")
else:
    print("No valid trading positions found for the refined strategy.")

"""## Summary:

### Data Analysis Key Findings

*   The dataset was loaded successfully with no missing values and the 'Date/Time' column already in the correct datetime format.
*   A simple moving average crossover strategy (40-day short window, 100-day long window) was implemented to generate trading signals.
*   Backtesting with an initial capital of 100,000 INR resulted in a final portfolio value of 61,006.00 INR.
*   The total profit/loss from the backtest was -38,994.00 INR, representing a percentage return of -38.99%.
*   The strategy was assessed as unprofitable based on the backtesting results.
*   Visualizations were generated showing the portfolio value over time and the portfolio drawdown over time.

### Insights or Next Steps

*   The current moving average crossover strategy resulted in a significant loss during the backtesting period. Further optimization of the moving average window lengths or exploring different trading strategies is necessary.
*   Implementing more sophisticated risk management techniques (e.g., stop-loss orders) and calculating additional performance metrics (e.g., Sharpe Ratio, max drawdown duration) would provide a more comprehensive evaluation of the strategy.

"""

: 